## **Team Reel Master Function -- Draft Working Version:**

- Python module that goes in parent directory (as .py file)
- runs when any new video or feedback is added (triggered by message in SQS queue for new video --> AWS Lambda function --> calls this function via API endpoint)
- calls all other TeamReel functions (ML ones by us) + gets human feedback from DB
- Adds video analysis (all of the above) to the video_feedback table in the DB

In [13]:
# Import modules/libraries we will use:

# Import external/third-party libraries we will use:
from dotenv import load_dotenv
import json
import numpy as np
import os
import psycopg2

# Import internal functions we need for TeamReel data infra, video and DB:
from data_infra.data_pipelines import get_next_video
from data_infra.postgresql_db_functions import get_feedback_for_user
from data_infra.postgresql_db_functions import get_feedback_for_video, get_video_info

# from audio_analysis.audio_functions import get_audio_from_video, get_transcript_from_audio, get_audio_sentiment_analysis, get_speed_of_speech, get_text_sentiment
# from audio_analysis.audio_functions import analyse_audio, remove_files

# Import functions we need from facial_analysis package

# Import functions we need from audio_analysis.background_noise module


# ----------------------------------------------------------------------------
# SETUP:

# Get access info from .env file:
load_dotenv()

# PostgreSQL DB info:
PG_DB_HOST = os.getenv("PG_DB_HOST")
PG_DB_PORT = os.getenv("PG_DB_PORT")
PG_DB_NAME = os.getenv("PG_DB_NAME")
PG_DB_USER = os.getenv("PG_DB_USER")
PG_DB_PW = os.getenv("PG_DB_PW")
PG_DB_URI = os.getenv("PG_DB_URI")

# Open a connection to our PostgreSQL DB:
pg_conn = psycopg2.connect(
    host = PG_DB_HOST,
    port = PG_DB_PORT,
    database = PG_DB_NAME,
    user = PG_DB_USER,
    password = PG_DB_PW
)

# Instantiate a cursor using this connection:
pg_cursor = pg_conn.cursor()


# ----------------------------------------------------------------------------


In [ ]:
# GET BASE MATERIALS: VIDEO, AUDIO, TRANSCRIPT:

# Get next video in line for analysis (recently uploaded by a user):
# (1) video_feedback dict = info about that video from our DB (video_id, etc.)
# (2) download .MP4 video file to project directory
video_feedback = get_next_video()

try:
    video_s3_key = video_info['video']['s3_key']
    video_filename = video_info['video']['s3_filename']
except KeyError:
    "KeyError: There is no information about this video in our database."

# Get audio from the video file:
get_audio_from_video(video_file=video_s3_filename)
audio_filename = 'audio.wav'

# ----------------------------------------------------------------------------
# SENTIMENT: AUDIO:

# Directory paths 
dirname = r"audio_chunks/"
path = r"text_chunks/"

# analyse_audio() function is the one function that does both: audio-sentiment analysis,
# and gets number of words per minute
data = analyse_audio() 

# Get the filename of the transcript
transcript_filename = 'outputfile.txt'
# Get the text in the transcript as a string
transcript_string = open(transcript_filename).read().replace("\n", " ")


# Add audio sentiment score to our video_feedback dict: 
video_feedback['sentiment_audio'] = data[0]['positive'] + data[0]['neutral'] # need to do some analysis and output 1 audio sentiment score
video_feedback['sentiment_audio_details'] = json.dumps(data[0])

# ----------------------------------------------------------------------------
# SPEED OF SPEECH: 

video_feedback['speaking_speed'] = round(data[2])


# ----------------------------------------------------------------------------
# SENTIMENT: VISUAL:

# [To add:] Facial centering: Call Chris Huskey's master function and get results
visual_sentiment_results = [To add]

# Add visual sentiment score to our video_feedback dict: 
video_feedback['sentiment_visual'] = ?? # need to do some analysis and output 1 visual sentiment score
video_feedback['sentiment_visual_details'] = visual_sentiment_results

# ----------------------------------------------------------------------------
# BACKGROUND NOISE:

# [To add:] Background noise: Call Chris Howell's master function and get results
# add to our video_feedback dict: video_feedback['']

# ----------------------------------------------------------------------------
# ADD VIDEO ANALYSIS RESULTS TO DB:

# [To add:] Add all of the above info for this video to the videos_feedback table in our DB

# pseudocode:
# if [video is in the video_feedback table already]:
#   Update info in the videos_feedback table
# else:
#   Add info to the video feedback table

# ----------------------------------------------------------------------------

# To add: Make Plotly visualizations so Web can display them super easily?

# ----------------------------------------------------------------------------
# REMOVE STORED FILES:

# To add: Remove local files (video, audio, transcript, audio chunks, (text chunks?))


In [ ]:
# {'video': {'video_id': 134,
#   'title': 'Kyla Oyamot',
#   's3_key': 'videos/ALPACAVID-i7swK-Wzc.webm',
#   's3_filename': 'ALPACAVID-i7swK-Wzc.webm',
#   'created_at': datetime.datetime(2020, 5, 5, 16, 22, 56, 852000, tzinfo=psycopg2.tz.FixedOffsetTimezone(offset=0, name=None)),
#   'updated_at': datetime.datetime(2020, 5, 5, 16, 22, 56, 852000, tzinfo=psycopg2.tz.FixedOffsetTimezone(offset=0, name=None))},
#  'prompt': {'prompt_id': 65,
#   'question': 'What is your first impression of Labs'},
#  'user': {'user_id': 185,
#   'first_name': 'Kyla',
#   'last_name': 'O',
#   'name': 'Kyla O',
#   'username': 'kylao'}}

In [12]:
# # Close the cursor:
# pg_cursor.close()

# # Close the connection:
# pg_conn.close()

In [ ]:
# id SERIAL PRIMARY KEY, 
# FOREIGN KEY (video_id) NOT NULL REFERENCES videos (id), 
# FOREIGN KEY (video_url) NOT NULL REFERENCES videos (video_url), 

# overall_performance NUMERIC, 

# sentiment_visual NUMERIC, 
# sentiment_visual_details JSONB, 
# sentiment_audio NUMERIC, 
# sentiment_audio_details JSONB, 
# sentiment_smile_frequency NUMERIC, 

# speaking_vocabulary NUMERIC, 
# speaking_speed NUMERIC, 

# background_noise NUMERIC, 

# appearance_facial_centering NUMERIC, 

# human_overall_performance NUMERIC, 
# human_delivery_and_presentation NUMERIC, 
# human_response_quality NUMERIC, 
# human_audio_quality NUMERIC, 
# human_visual_environment NUMERIC

In [ ]:
# - human score: overall rating
# - human score: delivery and presentation
# - human score: response quality
# - human score: audio quality
# - human score: visual environment
# - ML score: attitude
# - ML score: speaking speed
# - ML score: background noise
# - ML score: facial centering and posture

In [6]:
import datetime
import numpy as np
import pandas as pd

In [138]:
video_analysis = {'video': {'video_id': 162, 'title': 'DS testing 001', 's3_key': 'videos/ALPACAVID-Ut18n8aEt.mp4', 's3_filename': 'ALPACAVID-Ut18n8aEt.mp4', 'created_at': datetime.datetime(2020, 6, 8, 21, 46, 43, 703000, tzinfo=psycopg2.tz.FixedOffsetTimezone(offset=0, name=None)), 'updated_at': datetime.datetime(2020, 6, 8, 21, 46, 43, 703000, tzinfo=psycopg2.tz.FixedOffsetTimezone(offset=0, name=None))}, 'prompt': {'prompt_id': 85, 'question': 'tell me '}, 'user': {'user_id': 201, 'first_name': 'Christopher', 'last_name': 'Huskey', 'name': 'Christopher Huskey', 'username': 'chrishuskey'}, 'video_analysis': {'sentiment_visual': 0.6382284265482008, 'sentiment_visual_details': '{"emotions": {"sad": 0.4816375482887931, "calm": 0.8443668165181737, "fear": 0.9012952623858596, "angry": 0.031246441854258622, "happy": 0.45286566659565175, "confused": 0.163269892703233, "disgusted": 0.9995419575080721, "surprised": 0.7591465415994776}}', 'sentiment_audio': 0.9767861915758042, 'sentiment_audio_details': '{"positive": 0.75, "negative": 0.25}', 'speaking_speed': 96.55937846836848, 'background_noise': 3.1992346762511388, 'appearance_facial_centering': 3.935142172100987}}

In [139]:
video_analysis.keys()

dict_keys(['video', 'prompt', 'user', 'video_analysis'])

In [140]:
video_analysis

{'video': {'video_id': 162,
  'title': 'DS testing 001',
  's3_key': 'videos/ALPACAVID-Ut18n8aEt.mp4',
  's3_filename': 'ALPACAVID-Ut18n8aEt.mp4',
  'created_at': datetime.datetime(2020, 6, 8, 21, 46, 43, 703000, tzinfo=psycopg2.tz.FixedOffsetTimezone(offset=0, name=None)),
  'updated_at': datetime.datetime(2020, 6, 8, 21, 46, 43, 703000, tzinfo=psycopg2.tz.FixedOffsetTimezone(offset=0, name=None))},
 'prompt': {'prompt_id': 85, 'question': 'tell me '},
 'user': {'user_id': 201,
  'first_name': 'Christopher',
  'last_name': 'Huskey',
  'name': 'Christopher Huskey',
  'username': 'chrishuskey'},
 'video_analysis': {'sentiment_visual': 0.6382284265482008,
  'sentiment_visual_details': '{"emotions": {"sad": 0.4816375482887931, "calm": 0.8443668165181737, "fear": 0.9012952623858596, "angry": 0.031246441854258622, "happy": 0.45286566659565175, "confused": 0.163269892703233, "disgusted": 0.9995419575080721, "surprised": 0.7591465415994776}}',
  'sentiment_audio': 0.9767861915758042,
  'senti

In [141]:
video_s3_key = video_analysis['video']['s3_key']
video_s3_key

'videos/ALPACAVID-Ut18n8aEt.mp4'

In [142]:
video_id = video_analysis['video']['video_id']
video_id

162

In [143]:
video_filename = video_analysis['video']['s3_filename']
video_filename

'ALPACAVID-Ut18n8aEt.mp4'

In [144]:
query = f"SELECT * FROM videos_feedback WHERE video_id = {video_id}"
pg_cursor.execute(query)
results = pg_cursor.fetchall()
column_names = [column_name[0] for column_name in pg_cursor.description]
feedback_dataframe = pd.DataFrame(data=results, columns=column_names)
feedback_dataframe

,id,video_id,overall_performance,delivery_and_presentation,response_quality,audio_quality,visual_environment,attitude,sentiment_visual,sentiment_visual_details,...,background_visual_environment,background_noise,appearance_facial_centering,appearance_posture,appearance_gesticulation,human_overall_performance,human_delivery_and_presentation,human_response_quality,human_audio_quality,human_visual_environment
0,22,162,2.076,4.308,0.368,0.915,3.942,1.391,3.483,"{'emotions': {'sad': 0.14275693208309037, 'cal...",...,0.000,1.351,3.434,0.000,0.000,3.400,3.000,3.200,2.800,2.200


In [145]:
# IF EXISTS (SELECT 1 FROM videos_feedback WHERE video_id = 164) 
# 	BEGIN
# 		SELECT * FROM videos_feedback WHERE video_id = 164
# 	END

# ELSE
# 	BEGIN
# 		SELECT * FROM videos_feedback WHERE video_id = 201
# 	END

In [146]:
# video_id = 164
# # video_id = 201

In [147]:
# SENTIMENT ANALYSIS:

# VISUAL SENTIMENT:

# [?? To add: Facial centering: Call Chris Huskey's master function and get results ??]
# visual_sentiment_results = [?? To add ??]

# Values for our DB videos_feedback table:
sentiment_visual = np.random.uniform(0, 5)  # [?? To do: REMOVE this ??]
sentiment_visual_details_fake = {
    "emotions": {
        "sad": 0.4816375482887931,
        "calm": 0.8443668165181737,
        "fear": 0.9012952623858596,
        "angry": 0.031246441854258622,
        "happy": 0.45286566659565175,
        "confused": 0.163269892703233,
        "disgusted": 0.9995419575080721,
        "surprised": 0.7591465415994776
    }
}
sentiment_visual_details = json.dumps(sentiment_visual_details_fake)

# AUDIO AND TEXT SENTIMENT:

# audio_sentiment = get_audio_sentiment_analysis(audio_filename=audio_filename)
# text_sentiment = get_text_sentiment(file=transcript_filename)

# Values for our DB videos_feedback table:
sentiment_audio = np.random.uniform(0, 5)  # [?? To do: REMOVE this ??]
sentiment_audio_details_fake = {"neutral": 0.8840937553392264, 
                                 "negative": 0.6134209574862143, 
                                 "positive": 0.10086583590753595}
sentiment_audio_details = json.dumps(sentiment_audio_details_fake)


# ----------------------------------------------------------------------------
# SPEAKING SPEED:

# speaking_speed = get_speed_of_speech(transcript_filename=transcript_filename,
#                                       audio_filename=audio_filename)

# Values for our DB videos_feedback table:
speaking_speed = np.random.uniform(0, 5)  # [?? To do: REMOVE this ??]


# ----------------------------------------------------------------------------
# BACKGROUND NOISE:

# [To add:] Background noise: Call Chris Howell's master function and get results
# background_noise_score = [?? To add ??]

# Values for our DB videos_feedback table:
background_noise = np.random.uniform(0, 5)  # [?? To do: REMOVE this ??]


# ----------------------------------------------------------------------------
# APPEARANCE: FACIAL CENTERING:

# [To add:] Facial alignment and centering: Call Chris Huskey's master function
# and get results
# appearance_facial_centering = [?? To add ??]

# Values for our DB videos_feedback table:
appearance_facial_centering = np.random.uniform(0, 5)  # [?? To do: REMOVE this ??]

In [148]:
# HUMAN FEEDBACK:

# Get all human feedback on this video from our DB:
human_feedback_for_video = get_feedback_for_video(video_id=video_id)

# If there is no human feedback for this video yet, set all human feedback items to 0 in our DB
# (in our DB, 0 indicates no value yet):
if human_feedback_for_video.shape[0] < 1:
    human_overall_performance = 0
    human_delivery_and_presentation = 0
    human_response_quality = 0
    human_audio_quality = 0
    human_visual_environment = 0

# If there is human feedback for this user, get the average ratings:
else:
    human_overall_performance = human_feedback_for_video['overall_performance'].mean()
    human_delivery_and_presentation = human_feedback_for_video['delivery_and_presentation'].mean()
    human_response_quality = human_feedback_for_video['response_quality'].mean()
    human_audio_quality = human_feedback_for_video['audio_quality'].mean()
    human_visual_environment = human_feedback_for_video['visual_environment'].mean()

In [149]:
human_feedback_for_video

,id,post,video_id,owner_id,created_at,updated_at,viewed,overall_performance,delivery_and_presentation,response_quality,audio_quality,visual_environment
0,202,Great -- no comments!,162,216,2020-06-07 23:01:14.616412+00:00,2020-06-07 23:01:14.616412+00:00,False,3,4,2,3,2
1,203,Great -- no comments!,162,227,2020-06-10 23:01:14.616412+00:00,2020-06-10 23:01:14.616412+00:00,False,4,4,4,3,2
2,205,Great -- no comments!,162,215,2020-06-05 23:01:14.616412+00:00,2020-06-05 23:01:14.616412+00:00,False,2,4,4,2,2
3,206,Great -- no comments!,162,198,2020-06-03 23:01:14.616412+00:00,2020-06-03 23:01:14.616412+00:00,False,4,2,2,2,1
4,204,Great -- no comments!,162,197,2020-06-05 23:01:14.616412+00:00,2020-06-05 23:01:14.616412+00:00,False,4,1,4,4,4


In [150]:
human_overall_performance, human_delivery_and_presentation, human_response_quality, human_audio_quality, human_visual_environment

(3.4, 3.0, 3.2, 2.8, 2.2)

In [151]:
# Set values:
video_id = video_id
overall_performance = np.random.uniform(0, 5)  # [?? To do: REMOVE this ??]
delivery_and_presentation = 0
response_quality = human_response_quality
audio_quality = 0
visual_environment = 0
attitude = 0
sentiment_visual = sentiment_visual
sentiment_visual_details = sentiment_visual_details
sentiment_audio = sentiment_audio
sentiment_audio_details = sentiment_audio_details
speaking_confidence = 0
speaking_volume = 0
speaking_vocabulary = 0
speaking_speed = speaking_speed
speaking_filler_words = 0
background_visual_environment = 0
background_noise = background_noise
appearance_facial_centering = appearance_facial_centering
appearance_posture = 0
appearance_gesticulation = 0
human_overall_performance = human_overall_performance
human_delivery_and_presentation = human_delivery_and_presentation
human_response_quality = human_response_quality
human_audio_quality = human_audio_quality
human_visual_environment = human_visual_environment

In [152]:
# Dictionary of values to insert --> these values go into our SQL queries below:
values_to_insert = {
    'video_id': video_id, 
    'overall_performance': overall_performance, 
    'delivery_and_presentation': delivery_and_presentation, 
    'response_quality': response_quality, 
    'audio_quality': audio_quality, 
    'visual_environment': visual_environment, 
    'attitude': attitude, 
    'sentiment_visual': sentiment_visual, 
    'sentiment_visual_details': sentiment_visual_details, 
    'sentiment_audio': sentiment_audio, 
    'sentiment_audio_details': sentiment_audio_details, 
    'speaking_confidence': speaking_confidence, 
    'speaking_volume': speaking_volume, 
    'speaking_vocabulary': speaking_vocabulary, 
    'speaking_speed': speaking_speed, 
    'speaking_filler_words': speaking_filler_words, 
    'background_visual_environment': background_visual_environment, 
    'background_noise': background_noise, 
    'appearance_facial_centering': appearance_facial_centering, 
    'appearance_posture': appearance_posture, 
    'appearance_gesticulation': appearance_gesticulation, 
    'human_overall_performance': human_overall_performance, 
    'human_delivery_and_presentation': human_delivery_and_presentation, 
    'human_response_quality': human_response_quality, 
    'human_audio_quality': human_audio_quality, 
    'human_visual_environment': human_visual_environment
}

In [153]:
# Check if record for this video exists in the videos_feedback table yet, 
# and UPDATE or CREATE the record accordingly:
query = f"SELECT EXISTS (SELECT * FROM videos_feedback WHERE video_id = {video_id})"
pg_cursor.execute(query)
exists_in_vf_table = pg_cursor.fetchall()[0][0]

if exists_in_vf_table:
    print('yes, update it in videos_feedback table')  # [?? To do: REMOVE this ??]
    # Update record in videos_feedback table:
    pg_cursor.execute(
        """
        UPDATE videos_feedback 
        SET 
            overall_performance = %(overall_performance)s, 
            delivery_and_presentation = %(delivery_and_presentation)s, 
            response_quality = %(response_quality)s, 
            audio_quality = %(audio_quality)s, 
            visual_environment = %(visual_environment)s, 
            attitude = %(attitude)s, 
            sentiment_visual = %(sentiment_visual)s, 
            sentiment_visual_details = %(sentiment_visual_details)s, 
            sentiment_audio = %(sentiment_audio)s, 
            sentiment_audio_details = %(sentiment_audio_details)s, 
            speaking_confidence = %(speaking_confidence)s, 
            speaking_volume = %(speaking_volume)s, 
            speaking_vocabulary = %(speaking_vocabulary)s, 
            speaking_speed = %(speaking_speed)s, 
            speaking_filler_words = %(speaking_filler_words)s, 
            background_visual_environment = %(background_visual_environment)s, 
            background_noise = %(background_noise)s, 
            appearance_facial_centering = %(appearance_facial_centering)s, 
            appearance_posture = %(appearance_posture)s, 
            appearance_gesticulation = %(appearance_gesticulation)s, 
            human_overall_performance = %(human_overall_performance)s, 
            human_delivery_and_presentation = %(human_delivery_and_presentation)s, 
            human_response_quality = %(human_response_quality)s, 
            human_audio_quality = %(human_audio_quality)s, 
            human_visual_environment = %(human_visual_environment)s
        WHERE video_id = %(video_id)s
        """, 
        values_to_insert
    )
else:
    print('no, insert into videos_feedback table')  # [?? To do: REMOVE this ??]
    # Create record in videos_feedback table:
    pg_cursor.execute(
        """
        INSERT INTO videos_feedback(??) 
        VALUES (%(overall_performance)s, 
                %(delivery_and_presentation)s, 
                %(response_quality)s, 
                %(audio_quality)s, 
                %(visual_environment)s, 
                %(attitude)s, 
                %(sentiment_visual)s, 
                %(sentiment_visual_details)s, 
                %(sentiment_audio)s, 
                %(sentiment_audio_details)s, 
                %(speaking_confidence)s, 
                %(speaking_volume)s, 
                %(speaking_vocabulary)s, 
                %(speaking_speed)s, 
                %(speaking_filler_words)s, 
                %(background_visual_environment)s, 
                %(background_noise)s, 
                %(appearance_facial_centering)s, 
                %(appearance_posture)s, 
                %(appearance_gesticulation)s, 
                %(human_overall_performance)s, 
                %(human_delivery_and_presentation)s, 
                %(human_response_quality)s, 
                %(human_audio_quality)s, 
                %(human_visual_environment)s
                ) 
        WHERE video_id = %(video_id)s
        """, 
    values_to_insert
    )

pg_conn.commit()

yes, update it in videos_feedback table


In [154]:
query = f"SELECT * FROM videos_feedback WHERE video_id = {video_id}"
pg_cursor.execute(query)
results = pg_cursor.fetchall()
column_names = [column_name[0] for column_name in pg_cursor.description]
feedback_dataframe = pd.DataFrame(data=results, columns=column_names)
feedback_dataframe

,id,video_id,overall_performance,delivery_and_presentation,response_quality,audio_quality,visual_environment,attitude,sentiment_visual,sentiment_visual_details,...,background_visual_environment,background_noise,appearance_facial_centering,appearance_posture,appearance_gesticulation,human_overall_performance,human_delivery_and_presentation,human_response_quality,human_audio_quality,human_visual_environment
0,22,162,1.106,0.000,3.200,0.000,0.000,0.000,1.752,"{'emotions': {'sad': 0.4816375482887931, 'calm...",...,0.000,1.378,3.377,0.000,0.000,3.400,3.000,3.200,2.800,2.200


In [5]:
column_names

['id',
 'video_id',
 'overall_performance',
 'delivery_and_presentation',
 'response_quality',
 'audio_quality',
 'visual_environment',
 'attitude',
 'sentiment_visual',
 'sentiment_visual_details',
 'sentiment_audio',
 'sentiment_audio_details',
 'speaking_confidence',
 'speaking_volume',
 'speaking_vocabulary',
 'speaking_speed',
 'speaking_filler_words',
 'background_visual_environment',
 'background_noise',
 'appearance_facial_centering',
 'appearance_posture',
 'appearance_gesticulation',
 'human_overall_performance',
 'human_delivery_and_presentation',
 'human_response_quality',
 'human_audio_quality',
 'human_visual_environment']

In [ ]:
['video_id',
 'overall_performance',
 'delivery_and_presentation',
 'response_quality',
 'audio_quality',
 'visual_environment',
 'attitude',
 'sentiment_visual',
 'sentiment_visual_details',
 'sentiment_audio',
 'sentiment_audio_details',
 'speaking_confidence',
 'speaking_volume',
 'speaking_vocabulary',
 'speaking_speed',
 'speaking_filler_words',
 'background_visual_environment',
 'background_noise',
 'appearance_facial_centering',
 'appearance_posture',
 'appearance_gesticulation',
 'human_overall_performance',
 'human_delivery_and_presentation',
 'human_response_quality',
 'human_audio_quality',
 'human_visual_environment']

In [2]:
video_id = 165

In [12]:
query = f"SELECT * FROM videos_feedback WHERE video_id = {video_id}"
pg_cursor.execute(query)
results = pg_cursor.fetchall()
column_names = [column_name[0] for column_name in pg_cursor.description]
feedback_dataframe = pd.DataFrame(data=results, columns=column_names)
feedback_dataframe

,id,video_id,overall_performance,delivery_and_presentation,response_quality,audio_quality,visual_environment,attitude,sentiment_visual,sentiment_visual_details,...,background_visual_environment,background_noise,appearance_facial_centering,appearance_posture,appearance_gesticulation,human_overall_performance,human_delivery_and_presentation,human_response_quality,human_audio_quality,human_visual_environment
0,19,165,1.906,0.000,2.500,0.000,0.000,0.000,4.827,"{'emotions': {'sad': 0.4816375482887931, 'calm...",...,0.000,0.756,0.404,0.000,0.000,2.500,3.000,2.500,2.000,2.667


In [14]:
len(feedback_dataframe.columns)

27

In [14]:
video_id = 162

In [15]:
pg_cursor.execute(f"SELECT * FROM videos_feedback WHERE video_id = {video_id}")
results = pg_cursor.fetchall()
column_names = [column_name[0] for column_name in pg_cursor.description]
feedback_dataframe = pd.DataFrame(data=results, columns=column_names)
feedback_dataframe

,id,video_id,overall_performance,delivery_and_presentation,response_quality,audio_quality,visual_environment,attitude,sentiment_visual,sentiment_visual_details,...,background_visual_environment,background_noise,appearance_facial_centering,appearance_posture,appearance_gesticulation,human_overall_performance,human_delivery_and_presentation,human_response_quality,human_audio_quality,human_visual_environment
0,22,162,1.106,0.000,3.200,0.000,0.000,0.000,1.752,"{'emotions': {'sad': 0.4816375482887931, 'calm...",...,0.000,1.378,3.377,0.000,0.000,3.400,3.000,3.200,2.800,2.200


In [16]:
pg_cursor.execute(f"SELECT * FROM videos WHERE id = {video_id}")
results = pg_cursor.fetchall()
column_names = [column_name[0] for column_name in pg_cursor.description]
feedback_dataframe = pd.DataFrame(data=results, columns=column_names)
feedback_dataframe

,id,owner_id,title,description,created_at,updated_at,video_url,prompt_id,thumbnail
0,162,201,DS testing 001,DS test! 001,2020-06-08 21:46:43.703000+00:00,2020-06-08 21:46:43.703000+00:00,videos/ALPACAVID-Ut18n8aEt.mp4,85,None
